### Pipeline de Ingeniería de Características

En este notebook, prepararé mi dataset para predecir **`Puntaje_Credito`** con un procedimiento estructurado de ingeniería de características. Aquí va mi pipeline paso a paso:

1. **Eliminación de columnas innecesarias**:  
   - Elimino **`ID`**, **`Customer_ID`**, **`Mes`**, **`Ingreso_Anual`**, **`Ratio_Utilizacion_Credito`**, **`Num_Cuentas_Bancarias`** y **`Num_Prestamos`** por baja relevancia o redundancia (según correlaciones e importancias).  
   - También quito **`Nombre`** (sin valor predictivo) y **`Tipo_Prestamo`** (redundante con **`Num_Prestamos`**, y sus detalles no son clave para el puntaje).

2. **Ajuste de la columna `Edad`**:  
   - Analizo profundamente **`Edad`** (14 a 56 años).  
   - Me parece poco creíble que menores de 18 años tengan productos financieros o historial crediticio, ya que es inusual y muchas leyes limitan su acceso a servicios complejos como cuentas múltiples, tarjetas o préstamos.  
   - Elimino todos los registros de clientes menores de 18 años tras confirmar su incoherencia.

3. **Filtrado de clientes sin cuentas bancarias**:  
   - En **`Num_Cuentas_Bancarias`**, considero que un valor de 0 podría ser un error en los datos, porque sin cuentas bancarias no deberían tener productos financieros.  
   - Elimino todos los registros con 0 cuentas bancarias y luego descarto la columna como parte del paso 1.

4. **Filtrado de clientes sin préstamos**:  
   - En **`Num_Prestamos`**, quito registros con 0 préstamos, ya que estos clientes no tienen actividad crediticia relevante para mi modelo.  
   - Después, elimino la columna como planeado en el paso 1.

5. **Procesamiento de `Comportamiento_Pago`**:  
   - Verifico los valores únicos de **`Comportamiento_Pago`**.  
   - Los traduzco al español, defino su significado y aplico **OneHotEncoder** para convertirla en columnas binarias.

6. **Transformación de `Mezcla_Crediticia`**:  
   - Reviso **`Mezcla_Crediticia`** ("Good", "Standard", "Poor"), que mide la diversidad de productos financieros.  
   - Aplico **OneHotEncoder** para transformarla en variables binarias sin asumir orden.

7. **Codificación de `Pago_Minimo` y `Ocupacion`**:  
   - Uso **OneHotEncoder** en **`Pago_Minimo`** (relevante según análisis previo) y **`Ocupacion`** (profesión del cliente), generando columnas binarias útiles.

Con este pipeline, limpiaré el dataset de datos incoherentes o irrelevantes y transformaré las categorías en un formato óptimo para el modelo, asegurando que esté listo para predecir **`Puntaje_Credito`** de manera efectiva.

In [26]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import OneHotEncoder

In [27]:
# Ruta al archivo local
file_path = "../data/data_Result_EDA.csv"

In [28]:
# Cargar el dataset
df = pd.read_csv(file_path)

In [29]:
# Ajustar las opciones de visualización de pandas
pd.set_option('display.max_columns', None)  # Mostrar todas las columnas
pd.set_option('display.width', 1000)        # Ajustar el ancho de la salida para evitar cortes
pd.set_option('display.max_colwidth', None) # Mostrar el contenido completo de cada columna

In [30]:
# Mostrar las primeras filas
df.head(1)

,ID,ID_Cliente,Mes,Nombre,Edad,Numero_Seguro_Social,Ocupacion,Ingreso_Anual,Salario_Mensual,Num_Cuentas_Bancarias,Num_Tarjetas_Credito,Tasa_Interes,Num_Prestamos,Tipo_Prestamo,Retraso_Pago,Num_Pagos_Retrasados,Cambio_Limite_Credito,Num_Consultas_Credito,Mezcla_Crediticia,Deuda_Pendiente,Ratio_Utilizacion_Credito,Edad_Historial_Credito,Pago_Minimo,Total_Cuota_Mensual,Inversion_Mensual,Saldo_Mensual,Puntaje_Credito,Comportamiento_de_Pago
0,5634,3392,1,Aaron Maashoh,23.0,821000265.0,Scientist,19114.12,1824.843333,3.0,4.0,3.0,4.0,"Auto Loan, Credit-Builder Loan, Personal Loan, and Home Equity Loan",3.0,7.0,11.27,4.0,Good,809.98,26.82262,265.0,No,49.574949,21.46538,312.494089,Good,Alto gasto con pagos de pequeño valor


1. **Filtrar clientes sin cuentas bancarias**:  
   - Identificar registros donde **`Num_Cuentas_Bancarias`** sea 0.  
   - Eliminar esos registros del dataset.

In [31]:
# Contar la frecuencia de cada valor en Num_Cuentas_Bancarias
conteo_cuentas = df['Num_Cuentas_Bancarias'].value_counts().sort_index()

# Calcular el porcentaje
porcentaje = (conteo_cuentas / conteo_cuentas.sum() * 100).round(2)

# Crear un DataFrame con los resultados
tabla_cuentas = pd.DataFrame({
    'Número de Cuentas': conteo_cuentas.index,
    'Frecuencia': conteo_cuentas.values,
    'Porcentaje (%)': porcentaje.values
})

# Mostrar la tabla
print("Distribución del Número de Cuentas Bancarias:")
tabla_cuentas

Distribución del Número de Cuentas Bancarias:


,Número de Cuentas,Frecuencia,Porcentaje (%)
0,0.0,4417,4.42
1,1.0,4540,4.54
2,2.0,4340,4.34
3,3.0,12107,12.11
4,4.0,12343,12.34
5,5.0,12298,12.30
6,6.0,13175,13.18
7,7.0,12999,13.00
8,8.0,12940,12.94
9,9.0,5503,5.50


Al revisar la tabla, veo que 4,417 clientes (4.42%) tienen 0 cuentas bancarias. Esto me parece un dato extraño, porque si mi objetivo es predecir **`Puntaje_Credito`**, espero trabajar con personas que tengan actividad financiera activa. Sin cuentas bancarias, es poco probable que estos clientes tengan productos como préstamos, tarjetas de crédito o un historial crediticio relevante, ya que las cuentas son la base para manejar esos servicios. Ese 4.42% podría ser un error en los datos o casos irrelevantes, como personas sin vínculo real con el sistema financiero. Eliminarlos me asegura que mi dataset solo incluya clientes con actividad bancaria significativa, alineada con lo que quiero modelar. Además, el resto de la distribución (95.58%) muestra una variabilidad razonable (1 a 11 cuentas), lo que refuerza que mi foco debe estar en clientes activos.

In [32]:
# Eliminar registros con Num_Cuentas_Bancarias igual a 0
df = df[df['Num_Cuentas_Bancarias'] != 0]

# Verificar que se hayan eliminado (opcional)
print("Número de registros después de eliminar los que tienen 0 cuentas:", len(df))

Número de registros después de eliminar los que tienen 0 cuentas: 95583


2. **Filtrar clientes sin préstamos**:  
   - Identificar registros donde **`Num_Prestamos`** sea 0.  
   - Eliminar esos registros del dataset.

Ahora voy a filtrar mi dataset para quitar los registros donde **`Num_Prestamos`** es 0. Creo que esto tiene sentido porque estoy enfocada en predecir **`Puntaje_Credito`**, y los clientes sin préstamos no tienen actividad crediticia significativa que analizar. Si no tienen deudas ni créditos, no aportan información útil sobre cómo manejan productos financieros, que es lo que importa para mi modelo. Al eliminarlos, me aseguro de trabajar solo con datos relevantes para mi objetivo, manteniendo el foco en perfiles con historial crediticio activo.

In [33]:
# Eliminar registros con Num_Prestamos igual a 0
df = df[df['Num_Prestamos'] != 0]

# Verificar que se hayan eliminado (opcional)
print("Número de registros después de eliminar los que tienen 0 préstamos:", len(df))

Número de registros después de eliminar los que tienen 0 préstamos: 85118


3. **Eliminar columnas innecesarias**:  
   - Quitar del dataset las columnas: **`ID`**, **`Customer_ID`**, **`Mes`**, **`Ingreso_Anual`**, **`Ratio_Utilizacion_Credito`**, **`Num_Cuentas_Bancarias`**, **`Num_Prestamos`**, **`Nombre`**, **`Tipo_Prestamo`** y **`Numero_Seguro_Social`**.

In [34]:
# Lista de columnas a eliminar
columnas_a_eliminar = ['ID', 'ID_Cliente', 'Mes', 'Ingreso_Anual', 'Ratio_Utilizacion_Credito', 
                       'Num_Cuentas_Bancarias', 'Num_Prestamos', 'Nombre', 'Tipo_Prestamo', 'Numero_Seguro_Social']

# Eliminar las columnas del DataFrame
df = df.drop(columns=columnas_a_eliminar)

In [35]:
df.head()

,Edad,Ocupacion,Salario_Mensual,Num_Tarjetas_Credito,Tasa_Interes,Retraso_Pago,Num_Pagos_Retrasados,Cambio_Limite_Credito,Num_Consultas_Credito,Mezcla_Crediticia,Deuda_Pendiente,Edad_Historial_Credito,Pago_Minimo,Total_Cuota_Mensual,Inversion_Mensual,Saldo_Mensual,Puntaje_Credito,Comportamiento_de_Pago
0,23.0,Scientist,1824.843333,4.0,3.0,3.0,7.0,11.27,4.0,Good,809.98,265.0,No,49.574949,21.46538,312.494089,Good,Alto gasto con pagos de pequeño valor
1,23.0,Scientist,1824.843333,4.0,3.0,3.0,4.0,11.27,4.0,Good,809.98,266.0,No,49.574949,21.46538,284.629162,Good,Bajo gasto con pagos de gran valor
2,23.0,Scientist,1824.843333,4.0,3.0,3.0,7.0,11.27,4.0,Good,809.98,267.0,No,49.574949,21.46538,331.209863,Good,Bajo gasto con pagos de valor medio
3,23.0,Scientist,1824.843333,4.0,3.0,5.0,4.0,6.27,4.0,Good,809.98,268.0,No,49.574949,21.46538,223.451310,Good,Bajo gasto con pagos de pequeño valor
4,23.0,Scientist,1824.843333,4.0,3.0,6.0,4.0,11.27,4.0,Good,809.98,269.0,No,49.574949,21.46538,341.489231,Good,Alto gasto con pagos de valor medio


4. **Ajustar la columna `Edad`**:  
   - Analizar los valores de **`Edad`** para identificar cuántos clientes tienen menos de 18 años.  
   - Eliminar todos los registros donde **`Edad`** sea menor a 18.

In [36]:
# Filtrar los clientes menores de 18 años
menores_18 = df[df['Edad'] < 18]

# Calcular estadísticas básicas de los menores de 18
estadisticas_menores = menores_18.describe()

# Mostrar el número de menores y las estadísticas
print("Número de clientes menores de 18 años:", len(menores_18))
print("\nEstadísticas de los clientes menores de 18 años:")
estadisticas_menores

Número de clientes menores de 18 años: 5710

Estadísticas de los clientes menores de 18 años:


,Edad,Salario_Mensual,Num_Tarjetas_Credito,Tasa_Interes,Retraso_Pago,Num_Pagos_Retrasados,Cambio_Limite_Credito,Num_Consultas_Credito,Deuda_Pendiente,Edad_Historial_Credito,Total_Cuota_Mensual,Inversion_Mensual,Saldo_Mensual
count,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.00000,5710.000000,5710.000000,5710.000000
mean,15.577233,3146.949666,6.465149,20.288441,29.125044,16.883363,13.455445,8.811559,2098.009014,148.79352,129.378962,44.483333,311.925500
std,1.088438,2246.387040,2.057526,8.005340,15.523293,4.777156,6.600821,2.842858,1184.815473,72.37865,134.912154,27.570677,131.136859
min,14.000000,355.208333,3.000000,5.000000,0.000000,5.000000,0.500000,4.000000,2.040000,2.00000,6.442170,0.000000,0.103402
25%,15.000000,1396.905833,5.000000,15.000000,17.000000,14.000000,8.572500,7.000000,1320.310000,98.00000,45.441543,25.310288,245.810029
50%,16.000000,2594.606667,6.000000,19.000000,26.000000,17.000000,13.200000,8.000000,1933.900000,142.00000,84.004103,40.597975,291.088440
75%,17.000000,4640.715000,8.000000,27.000000,40.000000,20.000000,17.950000,11.000000,2698.320000,195.00000,175.631277,58.287311,360.798106
max,17.000000,12099.283333,10.000000,34.000000,62.000000,25.000000,29.970000,17.000000,4972.010000,393.00000,1762.000000,177.114154,1135.746342


### Mi análisis de los clientes menores de 18 años

Al revisar las estadísticas de los 5,710 clientes menores de 18 años, me sorprendí bastante. La edad promedio es 15.58 (rango 14-17), pero lo que realmente me parece increíble es que tengan un promedio de 6.47 tarjetas de crédito, una tasa de interés alta (20.29%), y una deuda pendiente de 2,098 en promedio. Incluso tienen un historial crediticio de 148.8 meses (¡más de 12 años!), lo cual es imposible para alguien de 14 o 15 años, ya que implicaría que empezaron a usar crédito desde los 2 o 3 años. Además, sus retrasos en pagos (promedio 29 días) y 16.88 pagos atrasados refuerzan que estos datos no son creíbles. El salario mensual promedio (3,146) también suena exagerado para menores, y aunque varía mucho (mínimo 355, máximo 12,099), no justifica esta actividad financiera tan compleja.

Esto me confirma que estos registros son anómalos, probablemente errores o datos mal registrados. Es inusual y poco realista que menores de 18 años tengan productos bancarios tan avanzados en esta magnitud, especialmente considerando las leyes que limitan su acceso. Por eso, creo firmemente que debo eliminarlos del dataset para mantener la coherencia en mi modelo de **`Puntaje_Credito`**.

In [37]:
# Eliminar registros con Edad menor a 18
df = df[df['Edad'] >= 17]

# Verificar el número de registros restantes (opcional)
print("Número de registros después de eliminar menores de 18:", len(df))

Número de registros después de eliminar menores de 18: 80913


### Verificación de la proporción entre Edad y Edad_Historial_Credito

Voy a calcular la proporción entre **`Edad`** y **`Edad_Historial_Credito`** porque quiero detectar registros incoherentes en mi dataset. 

Por ejemplo, si un cliente de 25 años tiene un historial crediticio de 20 años, eso implica que empezó a usar productos financieros a los 5 años, lo cual es muy poco creíble y probablemente un error. Para esto, dividiré la edad del cliente entre su historial crediticio (convertido a años dividiendo entre 12, ya que está en meses). 

Luego, estableceré un umbral razonable, como 1.5, para asegurarme de que el historial no sea demasiado largo respecto a la edad (por ejemplo, un historial de 10 años para alguien de 25 da una proporción de 2.5, que es aceptable). Filtraré mi dataset para quedarme solo con clientes mayores o iguales a 18 años y con una proporción mayor o igual al umbral, eliminando así casos anómalos. Esto me ayudará a mantener datos realistas y coherentes para predecir **`Puntaje_Credito`**.

In [38]:
# Calcular la proporción entre edad e historial crediticio (en años)
df['Proporcion_Edad_Historial'] = df['Edad'] / (df['Edad_Historial_Credito'] / 12)

# Definir un umbral razonable para la proporción
umbral_proporcion = 1.5

# Contar registros antes del filtrado (opcional, para seguimiento)
print("Número de registros antes del filtrado:", len(df))

# Filtrar clientes con edad >= 18 y proporción >= umbral, manejando posibles NaN
df = df[(df['Edad'] >= 18) & (df['Proporcion_Edad_Historial'] >= umbral_proporcion) & 
        (df['Proporcion_Edad_Historial'].notna())]

# Contar registros después del filtrado (opcional, para verificar)
print("Número de registros después del filtrado:", len(df))

# Eliminar la columna auxiliar 'Proporcion_Edad_Historial'
df = df.drop(columns=['Proporcion_Edad_Historial'], inplace=False)

Número de registros antes del filtrado: 80913
Número de registros después del filtrado: 55438


Antes de aplicar el filtro, tenía 80,913 registros en mi dataset. Después de eliminar los clientes con una proporción **`Edad` / `Edad_Historial_Credito`** menor a 1.5, me quedé con 55,438. Esto significa que perdí 25,475 registros, casi un 31.5% del total. Me sorprende un poco la cantidad, pero refuerza mi sospecha: muchos datos tenían historiales crediticios demasiado largos para las edades de los clientes, como si alguien de 20 años tuviera 18 años de crédito, lo cual no tiene sentido. Esto me dice que había bastantes registros incoherentes, y ahora mi dataset es más realista y confiable para predecir **`Puntaje_Credito`**.

5. **Procesar `Comportamiento_Pago`**:  
   - Verificar los valores únicos de **`Comportamiento_Pago`** con `.unique()`.  
   - Traducir los valores al español y documentar su significado.  
   - Aplicar **OneHotEncoder** para transformar la columna en variables binarias.

In [39]:
# Obtener los valores únicos de la columna 'Comportamiento_de_Pago'
valores_unicos = df['Comportamiento_de_Pago'].unique()

# Mostrar los valores únicos
print("Valores únicos en la columna 'Comportamiento_de_Pago':")
print(valores_unicos)

Valores únicos en la columna 'Comportamiento_de_Pago':
['Alto gasto con pagos de pequeño valor'
 'Bajo gasto con pagos de valor medio'
 'Bajo gasto con pagos de gran valor'
 'Alto gasto con pagos de valor medio'
 'Bajo gasto con pagos de pequeño valor'
 'Alto gasto con pagos de gran valor']


### Resumen del Análisis de "Comportamiento_de_Pago"

La columna **`Comportamiento_de_Pago`** describe cómo los clientes gastan (alto o bajo) y pagan (pequeño, medio o grande), ofreciendo seis categorías clave para entender su comportamiento financiero. Por ejemplo, "Alto gasto con pagos de pequeño valor" sugiere riesgo por posible acumulación de deuda, mientras que "Bajo gasto con pagos de gran valor" indica estabilidad y solvencia. Esto me permite segmentar a los clientes en perfiles de alto riesgo, responsables, intermedios o de bajo impacto, lo que es vital para predecir **`Puntaje_Credito`**. Como es categórica y sin orden lógico, usaré **OneHotEncoder** para convertirla en columnas binarias. Esto asegura que el modelo interprete cada categoría de forma independiente, sin sesgos, mejorando su precisión y permitiendo analizar qué patrones influyen más en las predicciones.

In [40]:
# Crear una instancia de OneHotEncoder
encoder = OneHotEncoder()

# Aplicar OneHotEncoder a la columna 'Comportamiento_de_Pago'
encoded_features = encoder.fit_transform(df[['Comportamiento_de_Pago']]).toarray()

# Obtener los nombres de las nuevas columnas generadas por el encoder
new_columns = [f"Comportamiento_{cat}" for cat in encoder.categories_[0]]

# Crear un DataFrame con las nuevas columnas
df_encoded = pd.DataFrame(encoded_features, columns=new_columns, index=df.index)

# Concatenar las nuevas columnas al DataFrame original
df = pd.concat([df, df_encoded], axis=1)

# Eliminar la columna original 'Comportamiento_de_Pago'
df.drop(columns=['Comportamiento_de_Pago'], inplace=True)

# Mostrar las primeras filas del DataFrame modificado
print("DataFrame después de aplicar OneHotEncoder:")
df.head()

DataFrame después de aplicar OneHotEncoder:


,Edad,Ocupacion,Salario_Mensual,Num_Tarjetas_Credito,Tasa_Interes,Retraso_Pago,Num_Pagos_Retrasados,Cambio_Limite_Credito,Num_Consultas_Credito,Mezcla_Crediticia,Deuda_Pendiente,Edad_Historial_Credito,Pago_Minimo,Total_Cuota_Mensual,Inversion_Mensual,Saldo_Mensual,Puntaje_Credito,Comportamiento_Alto gasto con pagos de gran valor,Comportamiento_Alto gasto con pagos de pequeño valor,Comportamiento_Alto gasto con pagos de valor medio,Comportamiento_Bajo gasto con pagos de gran valor,Comportamiento_Bajo gasto con pagos de pequeño valor,Comportamiento_Bajo gasto con pagos de valor medio
16,34.0,Engineer,12187.22,5.0,8.0,5.0,8.0,7.1,3.0,Good,1303.01,213.0,No,246.992319,168.413703,1043.315978,Good,0.0,1.0,0.0,0.0,0.0,0.0
17,34.0,Engineer,12187.22,5.0,8.0,13.0,6.0,7.1,3.0,Good,1303.01,214.0,No,246.992319,168.413703,998.869297,Good,0.0,1.0,0.0,0.0,0.0,0.0
18,34.0,Engineer,12187.22,5.0,8.0,8.0,7.0,11.1,3.0,Good,1303.01,215.0,No,246.992319,168.413703,715.741367,Good,0.0,1.0,0.0,0.0,0.0,0.0
19,34.0,Engineer,12187.22,5.0,8.0,8.0,5.0,9.1,3.0,Good,1303.01,216.0,No,246.992319,168.413703,426.513411,Good,0.0,0.0,0.0,0.0,0.0,1.0
20,34.0,Engineer,12187.22,5.0,8.0,10.0,5.0,7.1,3.0,Good,1303.01,217.0,No,246.992319,168.413703,810.782153,Good,0.0,0.0,0.0,1.0,0.0,0.0


6. **Transformar `Mezcla_Crediticia`**:  
   - Confirmar los valores únicos de **`Mezcla_Crediticia`** ("Good", "Standard", "Poor").  
   - Aplicar **OneHotEncoder** para convertirla en columnas binarias.

La columna **"Mezcla_Crediticia"** evalúa la diversidad y calidad de los productos financieros que un cliente ha utilizado, como tarjetas de crédito, préstamos o hipotecas. Una buena mezcla indica experiencia en el manejo de diferentes tipos de crédito y puede mejorar el puntaje crediticio, mientras que una mala mezcla sugiere posibles problemas financieros.

Aplico **OneHotEncoder** porque esta columna contiene categorías categóricas nominales (como "Good", "Standard", "Poor") sin un orden inherente. OneHotEncoder transforma estas categorías en columnas binarias independientes, permitiendo que el modelo interprete cada categoría de forma clara y sin suposiciones incorrectas. Esto mejora la precisión del modelo al capturar correctamente la relación entre la "Mezcla Crediticia" y la variable objetivo.

In [41]:
# Crear una instancia de OneHotEncoder
encoder = OneHotEncoder()

# Aplicar OneHotEncoder a la columna 'Mezcla_Crediticia'
encoded_features = encoder.fit_transform(df[['Mezcla_Crediticia']]).toarray()

# Obtener los nombres de las nuevas columnas generadas por el encoder
new_columns = [f"Mezcla_{cat}" for cat in encoder.categories_[0]]

# Crear un DataFrame con las nuevas columnas
df_encoded = pd.DataFrame(encoded_features, columns=new_columns, index=df.index)

# Concatenar las nuevas columnas al DataFrame original
df = pd.concat([df, df_encoded], axis=1)

# Eliminar la columna original 'Mezcla_Crediticia'
df.drop(columns=['Mezcla_Crediticia'], inplace=True)

# Mostrar las primeras filas del DataFrame modificado
print("DataFrame después de aplicar OneHotEncoder:")
df.head()

DataFrame después de aplicar OneHotEncoder:


,Edad,Ocupacion,Salario_Mensual,Num_Tarjetas_Credito,Tasa_Interes,Retraso_Pago,Num_Pagos_Retrasados,Cambio_Limite_Credito,Num_Consultas_Credito,Deuda_Pendiente,Edad_Historial_Credito,Pago_Minimo,Total_Cuota_Mensual,Inversion_Mensual,Saldo_Mensual,Puntaje_Credito,Comportamiento_Alto gasto con pagos de gran valor,Comportamiento_Alto gasto con pagos de pequeño valor,Comportamiento_Alto gasto con pagos de valor medio,Comportamiento_Bajo gasto con pagos de gran valor,Comportamiento_Bajo gasto con pagos de pequeño valor,Comportamiento_Bajo gasto con pagos de valor medio,Mezcla_Bad,Mezcla_Good,Mezcla_Standard
16,34.0,Engineer,12187.22,5.0,8.0,5.0,8.0,7.1,3.0,1303.01,213.0,No,246.992319,168.413703,1043.315978,Good,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
17,34.0,Engineer,12187.22,5.0,8.0,13.0,6.0,7.1,3.0,1303.01,214.0,No,246.992319,168.413703,998.869297,Good,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
18,34.0,Engineer,12187.22,5.0,8.0,8.0,7.0,11.1,3.0,1303.01,215.0,No,246.992319,168.413703,715.741367,Good,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
19,34.0,Engineer,12187.22,5.0,8.0,8.0,5.0,9.1,3.0,1303.01,216.0,No,246.992319,168.413703,426.513411,Good,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
20,34.0,Engineer,12187.22,5.0,8.0,10.0,5.0,7.1,3.0,1303.01,217.0,No,246.992319,168.413703,810.782153,Good,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


7. **Codificar `Pago_Minimo` y `Ocupacion`**:  
   - Revisar los valores únicos de **`Pago_Minimo`** y **`Ocupacion`**.  
   - Aplicar **OneHotEncoder** a ambas columnas para generar variables binarias.

In [42]:
# Paso 1: Revisar los valores únicos de Pago_Minimo y Ocupacion
print("Valores únicos de Pago_Minimo:")
print(df['Pago_Minimo'].unique())

print("\nValores únicos de Ocupacion:")
print(df['Ocupacion'].unique())

# Paso 2: Aplicar OneHotEncoder a ambas columnas
encoder = OneHotEncoder(sparse_output=False, drop='first')  # drop='first' evita multicolinealidad
encoded_data = encoder.fit_transform(df[['Pago_Minimo', 'Ocupacion']])
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(['Pago_Minimo', 'Ocupacion']))

# Combinar el DataFrame original con las columnas codificadas y eliminar las originales
df = df.drop(columns=['Pago_Minimo', 'Ocupacion']).reset_index(drop=True)
df = pd.concat([df, encoded_df], axis=1)

Valores únicos de Pago_Minimo:
['No' 'NM' 'Yes']

Valores únicos de Ocupacion:
['Engineer' 'Entrepreneur' 'Media_Manager' 'Journalist' 'Teacher'
 'Manager' 'Scientist' 'Lawyer' 'Mechanic' 'Writer' 'Accountant'
 'Musician' 'Architect' 'Developer' 'Doctor']


In [43]:
df.columns

Index(['Edad', 'Salario_Mensual', 'Num_Tarjetas_Credito', 'Tasa_Interes', 'Retraso_Pago', 'Num_Pagos_Retrasados', 'Cambio_Limite_Credito', 'Num_Consultas_Credito', 'Deuda_Pendiente', 'Edad_Historial_Credito', 'Total_Cuota_Mensual', 'Inversion_Mensual', 'Saldo_Mensual', 'Puntaje_Credito', 'Comportamiento_Alto gasto con pagos de gran valor', 'Comportamiento_Alto gasto con pagos de pequeño valor', 'Comportamiento_Alto gasto con pagos de valor medio', 'Comportamiento_Bajo gasto con pagos de gran valor', 'Comportamiento_Bajo gasto con pagos de pequeño valor', 'Comportamiento_Bajo gasto con pagos de valor medio', 'Mezcla_Bad', 'Mezcla_Good', 'Mezcla_Standard', 'Pago_Minimo_No', 'Pago_Minimo_Yes', 'Ocupacion_Architect', 'Ocupacion_Developer', 'Ocupacion_Doctor', 'Ocupacion_Engineer', 'Ocupacion_Entrepreneur', 'Ocupacion_Journalist', 'Ocupacion_Lawyer', 'Ocupacion_Manager', 'Ocupacion_Mechanic', 'Ocupacion_Media_Manager', 'Ocupacion_Musician', 'Ocupacion_Scientist', 'Ocupacion_Teacher',
      

In [44]:
# Definir el orden deseado de las columnas
orden_columnas = [
    # Variables numéricas originales
    'Edad', 'Salario_Mensual', 'Num_Tarjetas_Credito', 'Tasa_Interes', 
    'Retraso_Pago', 'Num_Pagos_Retrasados', 'Cambio_Limite_Credito', 
    'Num_Consultas_Credito', 'Deuda_Pendiente', 'Edad_Historial_Credito', 
    'Total_Cuota_Mensual', 'Inversion_Mensual', 'Saldo_Mensual',
    
    # Variables codificadas: Comportamiento_Pago (ordenadas por gasto y valor)
    'Comportamiento_Alto gasto con pagos de gran valor', 
    'Comportamiento_Alto gasto con pagos de valor medio', 
    'Comportamiento_Alto gasto con pagos de pequeño valor', 
    'Comportamiento_Bajo gasto con pagos de gran valor', 
    'Comportamiento_Bajo gasto con pagos de valor medio', 
    'Comportamiento_Bajo gasto con pagos de pequeño valor',
    
    # Variables codificadas: Mezcla_Crediticia (orden alfabético)
    'Mezcla_Bad', 'Mezcla_Good', 'Mezcla_Standard',
    
    # Variables codificadas: Pago_Minimo
    'Pago_Minimo_No', 'Pago_Minimo_Yes',
    
    # Variables codificadas: Ocupacion (orden alfabético)
    'Ocupacion_Architect', 'Ocupacion_Developer', 'Ocupacion_Doctor', 
    'Ocupacion_Engineer', 'Ocupacion_Entrepreneur', 'Ocupacion_Journalist', 
    'Ocupacion_Lawyer', 'Ocupacion_Manager', 'Ocupacion_Mechanic', 
    'Ocupacion_Media_Manager', 'Ocupacion_Musician', 'Ocupacion_Scientist', 
    'Ocupacion_Teacher', 'Ocupacion_Writer',
    
    # Variable objetivo al final
    'Puntaje_Credito'
]

# Reorganizar el DataFrame con el nuevo orden
df = df[orden_columnas]

# Verificar las columnas reorganizadas (opcional)
print("Columnas reorganizadas")

Columnas reorganizadas


In [45]:
df.head()

,Edad,Salario_Mensual,Num_Tarjetas_Credito,Tasa_Interes,Retraso_Pago,Num_Pagos_Retrasados,Cambio_Limite_Credito,Num_Consultas_Credito,Deuda_Pendiente,Edad_Historial_Credito,Total_Cuota_Mensual,Inversion_Mensual,Saldo_Mensual,Comportamiento_Alto gasto con pagos de gran valor,Comportamiento_Alto gasto con pagos de valor medio,Comportamiento_Alto gasto con pagos de pequeño valor,Comportamiento_Bajo gasto con pagos de gran valor,Comportamiento_Bajo gasto con pagos de valor medio,Comportamiento_Bajo gasto con pagos de pequeño valor,Mezcla_Bad,Mezcla_Good,Mezcla_Standard,Pago_Minimo_No,Pago_Minimo_Yes,Ocupacion_Architect,Ocupacion_Developer,Ocupacion_Doctor,Ocupacion_Engineer,Ocupacion_Entrepreneur,Ocupacion_Journalist,Ocupacion_Lawyer,Ocupacion_Manager,Ocupacion_Mechanic,Ocupacion_Media_Manager,Ocupacion_Musician,Ocupacion_Scientist,Ocupacion_Teacher,Ocupacion_Writer,Puntaje_Credito
0,34.0,12187.22,5.0,8.0,5.0,8.0,7.1,3.0,1303.01,213.0,246.992319,168.413703,1043.315978,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Good
1,34.0,12187.22,5.0,8.0,13.0,6.0,7.1,3.0,1303.01,214.0,246.992319,168.413703,998.869297,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Good
2,34.0,12187.22,5.0,8.0,8.0,7.0,11.1,3.0,1303.01,215.0,246.992319,168.413703,715.741367,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Good
3,34.0,12187.22,5.0,8.0,8.0,5.0,9.1,3.0,1303.01,216.0,246.992319,168.413703,426.513411,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Good
4,34.0,12187.22,5.0,8.0,10.0,5.0,7.1,3.0,1303.01,217.0,246.992319,168.413703,810.782153,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Good


In [46]:
import os

# Crear la carpeta 'data' si no existe
if not os.path.exists('../data'):
    os.makedirs('data')

# Guardar el DataFrame en un archivo CSV dentro de la carpeta 'data'
df.to_csv('../data/data_Ing_Caracteristicas.csv', index=False)

print("El archivo 'data_Ing_Caracteristicas.csv' ha sido guardado exitosamente en la carpeta 'data'.")

El archivo 'data_Ing_Caracteristicas.csv' ha sido guardado exitosamente en la carpeta 'data'.
